In [154]:
# Librería para comando de sistema
import os
# Librerías para manejo de datos
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# Librerías de aprendizaje automático.
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para construir un modelo con el algoritmo de regresión lineal
from sklearn.linear_model import LinearRegression
# Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Para sacar un reporte estadístico que podemos usar para determinar las importancia de las variables explicativas.
import statsmodels.api as sm 


##Import pipeline

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


In [155]:
# Se cargan los datos. 

train=pd.read_csv('laptop_data_test_unlabeled.csv')
test=pd.read_csv('laptop_data_train.csv')
print(train.shape)
print(test.shape)
train

(64, 14)
(1216, 14)


,id,Company,TypeName,Ram,Weight,TouchScreen,Ips,Ppi,Cpu_brand,HDD,SSD,Gpu_brand,Os,CalculatedPrice
0,1234,HP,Notebook,4,2.10,0,0,141.211998,Intel Core i3,1000.0,0,Intel,Windows,NaN
1,691,Lenovo,Notebook,4,1.70,0,0,157.350512,Intel Core i5,0.0,256,Intel,Windows,NaN
2,894,MSI,Gaming,16,2.43,0,1,254.671349,Intel Core i7,2000.0,512,Nvidia,Windows,NaN
3,766,Asus,Notebook,4,2.80,0,0,90.583402,Other Intel Processor,1000.0,0,Nvidia,Windows,NaN
4,739,Acer,Notebook,4,2.40,0,0,100.454670,Intel Core i3,500.0,0,Intel,Windows,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1088,HP,Notebook,4,1.29,0,0,276.053530,Other Intel Processor,0.0,0,Intel,Others,NaN
60,750,Lenovo,Netbook,4,1.40,1,0,135.094211,Other Intel Processor,0.0,128,Intel,Windows,NaN
61,581,Dell,Notebook,8,1.90,0,0,100.454670,Intel Core i5,500.0,0,Intel,Windows,NaN
62,348,Asus,2 in 1 Convertible,4,1.50,1,0,135.094211,Other Intel Processor,0.0,0,Intel,Windows,NaN


In [156]:
test.isna().sum()/len(test)

id             0.000000
Company        0.000000
TypeName       0.000000
Ram            0.000000
Weight         0.018914
Price          0.012336
TouchScreen    0.000000
Ips            0.000000
Ppi            0.009868
Cpu_brand      0.000000
HDD            0.013158
SSD            0.000000
Gpu_brand      0.000000
Os             0.000000
dtype: float64

In [157]:
train.isna().sum()/len(train)


id                 0.000000
Company            0.000000
TypeName           0.000000
Ram                0.000000
Weight             0.015625
TouchScreen        0.000000
Ips                0.000000
Ppi                0.000000
Cpu_brand          0.000000
HDD                0.000000
SSD                0.000000
Gpu_brand          0.000000
Os                 0.000000
CalculatedPrice    1.000000
dtype: float64

In [ ]:
train = train.dropna()

In [158]:
numerical = train.select_dtypes(include=np.number).columns.tolist()
numerical.remove("id")
numerical.remove("CalculatedPrice")
numerical

categoricas = train.select_dtypes(exclude = np.number).columns.tolist()
categoricas

['Company', 'TypeName', 'Cpu_brand', 'Gpu_brand', 'Os']

In [159]:
x = train[numerical+ categoricas]
y = train["CalculatedPrice"]
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
x_REAL_test = test[numerical+ categoricas]



# PIPELINE

In [160]:
numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("encoder", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(sparse=True, handle_unknown="ignore"))
])

preprocessing_pipeline = ColumnTransformer([
    ("numerical_preprocessor", numerical_pipeline, numerical),
    ("categorical_preprocessor", categorical_pipeline, categoricas)
])


In [161]:
#Todo en un pipeline

ML_model = LinearRegression()
complete_pipeline = Pipeline([
    ("preprocessor", preprocessing_pipeline),
    ("estimator", ML_model)
])

In [162]:
#Prediccion 

complete_pipeline.fit(x_train, y_train)
y_pred = complete_pipeline.predict(x_test)

print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

p1 = max(max(y_pred),max(y_test))
p2= min(min(y_pred),min(y_test))
plt.plot([p1,p1],[p1,p2],'b-')
plt.scatter(y_test,y_pred)

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: Input y contains NaN.